In [1]:
import numpy as np
import os
import tensorflow as tf
import tensorflow.keras as K


In [2]:
from keras.datasets import cifar10
from keras import initializers

import copy

In [3]:
use_colab = True
assert use_colab in [True, False]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [6]:
print ("Training data:")
print ("Number of examples: ", X_train.shape[0])
print ("Number of channels:",X_train.shape[3]) 
print ("Image size:", X_train.shape[1], X_train.shape[2])
print
print ("Test data:")
print ("Number of examples:", X_test.shape[0])
print ("Number of channels:", X_test.shape[3])
print ("Image size:", X_test.shape[1], X_test.shape[2]) 

print(X_train.shape, X_train.dtype)

Training data:
Number of examples:  50000
Number of channels: 3
Image size: 32 32
Test data:
Number of examples: 10000
Number of channels: 3
Image size: 32 32
(50000, 32, 32, 3) uint8


In [7]:

print ("mean before normalization:", np.mean(X_train)) 
print ("std before normalization:", np.std(X_train))

mean=[0,0,0]
std=[0,0,0]
newX_train = np.ones(X_train.shape)
newX_test = np.ones(X_test.shape)
for i in range(3):
    mean[i] = np.mean(X_train[:,:,:,i])
    std[i] = np.std(X_train[:,:,:,i])
    
for i in range(3):
    newX_train[:,:,:,i] = X_train[:,:,:,i] - mean[i]
    newX_train[:,:,:,i] = newX_train[:,:,:,i] / std[i]
    newX_test[:,:,:,i] = X_test[:,:,:,i] - mean[i]
    newX_test[:,:,:,i] = newX_test[:,:,:,i] / std[i]
        
    
X_train = newX_train
X_test = newX_test

print ("mean after normalization:", np.mean(X_train))
print ("std after normalization:", np.std(X_train))
print(X_train.max())

mean before normalization: 120.70756512369792
std before normalization: 64.1500758911213
mean after normalization: 4.91799193961621e-17
std after normalization: 0.9999999999999996
2.126789409516928


ResNet pretrained Model Load

In [8]:
## back born
resNet_base = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    # input_tensor=None,
    input_shape=(224,224,3),
    pooling='max',
    classes=10
)


AttributeError: ignored

In [ ]:
resNet_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
# load 한 모델의 레이어 숫자 
len(resNet_base.layers)

176

In [ ]:
# 로드한 모델 내 학습되는 레이어 숫자 출력
print(len(resNet_base.trainable_weights))

212


In [ ]:
# 전체 레이어 학습설정
resNet_base.trainable = False

In [ ]:
# 로드한 모델 내 학습되는 레이어 숫자 출력
print(len(resNet_base.trainable_weights))

0


In [ ]:
# 일부 layer trainable 설정

train_target_layer = 5

# 0 ~ 마지막 21번째 : 학습제외
for layer in resNet_base.layers[0:-train_target_layer]:
  layer.trainable = False



In [ ]:
# 로드한 모델 내 학습되는 레이어 숫자 출력
print(len(resNet_base.trainable_weights))

4


cifar10 model redefine

In [ ]:
## 모델
input_Layer = tf.keras.layers.Input(shape=(32,32,3))

# pretrained 모델 입력에 맞춰서 입력사이즈 변경
#x = tf.keras.layers.ZeroPadding2D( (96,96), input_shape=(32,32,3))(input_Layer)
x = tf.keras.layers.Lambda(lambda image: tf.image.resize(image, (224, 224)))(input_Layer)

x = resNet_base(x)
x=tf.keras.layers.Flatten()(x)
#x= tf.keras.layers.Dense(512, activation='relu')(x)
Out_Layer= tf.keras.layers.Dense(10, activation='softmax', kernel_initializer='he_normal')(x)
model = tf.keras.Model(inputs=[input_Layer], outputs=[Out_Layer])

In [ ]:
print(' 동결하기 전 conv_base를훈련되는 가중치의 수:',
      len(model.trainable_weights))

#resNet_base.trainable = True   ## True면 back의 초기값이 가지고있던 웨이트로해서 학습 False면 bakcborn뒤의 모델만 학습

print('conv_base를 동결한 후 훈련되는 가중치의 수:',
      len(model.trainable_weights))  ## dence layer가 2개 선언. 각 layer 마다 w,b 파라미터 2개가 있고, 2개의 레이어만 학습하므로 총 4개 파라미터만 학습


 동결하기 전 conv_base를훈련되는 가중치의 수: 2
conv_base를 동결한 후 훈련되는 가중치의 수: 2


In [ ]:
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 20,490
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
# the save point
if use_colab:
    checkpoint_dir ='/content/drive/My Drive/0003. AICC 자료/001. Study 자료/Project_Code/CIFAR-10/pretrained_resnet50/exp1'
else:
    checkpoint_dir = 'pretrained_resnet50/exp1'

if not os.path.isdir(checkpoint_dir):
        os.makedirs(checkpoint_dir)    

In [ ]:
model.compile(loss='categorical_crossentropy', 
                  optimizer='adam',
                  metrics='categorical_accuracy')

In [ ]:
import keras.utils

one_hot_y_train = keras.utils.to_categorical(y_train, num_classes=10)

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,
                                                 save_weights_only=True,
                                                 monitor='val_loss',
                                                 mode='auto',
                                                 save_best_only=True,
                                                 verbose=1)

In [ ]:
model.fit(X_train, one_hot_y_train, epochs=150, batch_size=1000, validation_split=0.2
          ,callbacks=[cp_callback])

Epoch 1/150
40/40 [==============================] - ETA: 0s - loss: 1.5867 - categorical_accuracy: 0.4500WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0103s vs `on_test_batch_end` time: 1.3009s). Check your callbacks.

Epoch 00001: val_loss did not improve from 1.27880
40/40 [==============================] - 66s 2s/step - loss: 1.5867 - categorical_accuracy: 0.4500 - val_loss: 1.5686 - val_categorical_accuracy: 0.4483
Epoch 2/150
40/40 [==============================] - ETA: 0s - loss: 1.5240 - categorical_accuracy: 0.4738
Epoch 00002: val_loss did not improve from 1.27880
40/40 [==============================] - 66s 2s/step - loss: 1.5240 - categorical_accuracy: 0.4738 - val_loss: 1.5115 - val_categorical_accuracy: 0.4752
Epoch 3/150
40/40 [==============================] - ETA: 0s - loss: 1.4743 - categorical_accuracy: 0.4932
Epoch 00003: val_loss did not improve from 1.27880
40/40 [==============================] - 66s 2s

In [ ]:
model.save('cifar10_pretrain_resnet50.h5')

Test 결과분석 - Confusion Matrix 

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

#     print(cm)

    fig, ax = plt.subplots(figsize=(7,7))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

In [ ]:
y_pred=model_1.predict_classes(x_test)
y_true=np.argmax(y_test,axis=1)

#Compute the confusion matrix
confusion_mtx=confusion_matrix(y_true,y_pred)

In [ ]:
class_names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# Plot non-normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes=class_names,
                      title='Confusion matrix, without normalization')

In [ ]:
# Plot normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')
# plt.show()